# Image sentiment
Now that I have labeled images I can create a model to train on them.

In [1]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.0.1
Torchvision Version:  0.2.1


In [2]:
# Batch size for training (change depending on how much memory you have)
batch_size = 200

# Number of epochs to train for
num_epochs = 8

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = False

In [3]:
import pandas as pd

file_path = '../Datasets/predictions.json'
tweets = pd.read_json(file_path, encoding="latin-1")

In [4]:
tweets = tweets[[3,5]]
tweets = tweets.groupby(3).mean()
tweets['path'] = tweets.index

Filter out tweets with broken images

In [5]:
from tqdm import tqdm
import pretrainedmodels.utils as utils

load_img = utils.LoadImage()
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

tweets['ok'] = 1

for index, row in tqdm(tweets.iterrows(), total=len(tweets)):
    try:
        img = load_img(row['path'])
        img = transform(img)
    except:
        tweets.at[index, 'ok'] = 0
        
tweets = tweets[tweets['ok'] == 1]

 29%|██▉       | 9549/32947 [01:22<03:10, 123.02it/s]C:\Users\Robin\Anaconda3\lib\site-packages\PIL\Image.py:952: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  ' expressed in bytes should be converted ' +
100%|██████████| 32947/32947 [04:37<00:00, 118.92it/s]


In [6]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_acc2 = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            running_corrects2 = 0
            
            t = tqdm(dataloaders[phase], mininterval=1, desc='-(' + phase + ')', leave=False)

            # Iterate over data.
            for inputs, labels in t:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    preds = model(inputs)
                    loss = criterion(preds, labels.squeeze_())
                    description = "Loss: " + str(loss.item())
                    t.set_description(description)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()   

                # statistics
                running_loss += loss.item() * inputs.size(0)
                for i in range(len(preds)):
                    pred = preds[i]
                    label = labels.data[i]
                    
                    if(torch.abs(pred-label) < 0.2):
                        running_corrects += 1
                        
                    if(pred < 0.5 and label < 0.5 or pred > 0.5 and label > 0.5):
                        running_corrects2 += 1
                    

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects / len(dataloaders[phase].dataset)
            epoch_acc2 = running_corrects2 / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f} Acc2: {:.4f}'.format(phase, epoch_loss, epoch_acc, epoch_acc2))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_acc2 = epoch_acc2
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:.4f}  Acc2: {:.4f}'.format(best_acc, best_acc2))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [7]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [8]:
def initialize_model(feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = models.alexnet(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.classifier[6].in_features
    model_ft.classifier[6] = nn.Linear(num_ftrs, 1)
    model_ft = nn.Sequential(
        model_ft,
        nn.Sigmoid()
    )
    input_size = 224

    return model_ft, input_size

# Initialize the model for this run
model_ft, input_size = initialize_model(feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

Sequential(
  (0): AlexNet(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace)
      (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (classifier): Sequential(
      (0): Dropout(p=0.5)
      (1): Linear(in_features=9216, out_features=4096, bias=True)
      (2): ReLU(inplace)
      (3): D

In [9]:
from torch.utils.data.dataset import Dataset

import codecs
import os
import torch

class LoadData(Dataset):
    def __init__(self, data):
        self.load_img = utils.LoadImage()
        self.transform = transforms.Compose([
            transforms.RandomResizedCrop(input_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        self.data = data.values

    def __getitem__(self, index):   
        path_img = self.data[index][1]
        sentiment = self.data[index][0]
        img = self.load_img(path_img)
        img = self.transform(img)

        sentiment = torch.tensor([sentiment], dtype=torch.float32)

        return img, sentiment

    def __len__(self):
        return len(self.data)

In [10]:
print("Initializing Datasets and Dataloaders...")

# Split dataset
train_data = tweets.sample(frac=0.8)
test_data = tweets.loc[~tweets.index.isin(train_data.index), :]

# Depending on what I want to classifier I can either use the default fold erstructure or need to use my own data loader.
image_datasets = {
    'train': LoadData(train_data),
    'val': LoadData(test_data)
}

# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True) for x in ['train', 'val']}

# Detect if we have a GPU available
useGPU = True
device = torch.device("cuda:0" if torch.cuda.is_available() and useGPU else "cpu")

Initializing Datasets and Dataloaders...


In [11]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 0.features.0.weight
	 0.features.0.bias
	 0.features.3.weight
	 0.features.3.bias
	 0.features.6.weight
	 0.features.6.bias
	 0.features.8.weight
	 0.features.8.bias
	 0.features.10.weight
	 0.features.10.bias
	 0.classifier.1.weight
	 0.classifier.1.bias
	 0.classifier.4.weight
	 0.classifier.4.bias
	 0.classifier.6.weight
	 0.classifier.6.bias


In [12]:
# Setup the loss fxn
criterion = nn.MSELoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 0/7
----------


train Loss: 0.0593 Acc: 0.5951 Acc2: 0.7444


val Loss: 0.0539 Acc: 0.5998 Acc2: 0.7638

Epoch 1/7
----------


train Loss: 0.0550 Acc: 0.6045 Acc2: 0.7685


val Loss: 0.0526 Acc: 0.6010 Acc2: 0.7689

Epoch 2/7
----------


...

val Loss: 0.0510 Acc: 0.6117 Acc2: 0.7720

Epoch 6/7
----------


train Loss: 0.0521 Acc: 0.6071 Acc2: 0.7774


val Loss: 0.0510 Acc: 0.6038 Acc2: 0.7718

Epoch 7/7
----------


train Loss: 0.0519 Acc: 0.6056 Acc2: 0.7772


val Loss: 0.0508 Acc: 0.6086 Acc2: 0.7721

Training complete in 34m 42s
Best val Acc: 0.6117  Acc2: 0.7720


Save model.

In [13]:
torch.save(model_ft.state_dict(), 'final_model')

## Sarcasm detection

In [32]:
from torch.autograd import Variable

def predict(data_iterator, model):
    model.eval()
    t = tqdm(data_iterator, mininterval=1, desc='-(Prediction)', leave=False)
    
    test_data['sentiment'] = 0.5
    values = test_data.values
    counter = 0
    
    for batch in t:
        image, score = batch
        if torch.cuda.is_available() and useGPU:
            image = Variable(image.cuda())
            
        pred = model(image)
        pred = pred.cpu()
        for p in pred:
            values[counter][3] = p.item()
            counter += 1
        
    return pd.DataFrame(values)

In [44]:
predictions = predict(dataloaders_dict['val'], model_ft)






-(Prediction):   0%|          | 0/33 [00:00<?, ?it/s]C:\Users\Robin\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys





-(Prediction):   3%|▎         | 1/33 [00:01<00:36,  1.14s/it]




-(Prediction):   9%|▉         | 3/33 [00:02<00:31,  1.07s/it]




-(Prediction):  15%|█▌        | 5/33 [00:04<00:27,  1.02it/s]




-(Prediction):  21%|██        | 7/33 [00:06<00:25,  1.03it/s]




-(Prediction):  27%|██▋       | 9/33 [00:08<00:22,  1.08it/s]




-(Prediction):  33%|███▎      | 11/33 [00:09<00:19,  1.12it/s]




-(Prediction):  39%|███▉      | 13/33 [00:11<00:17,  1.12it/s]




-(Prediction):  45%|████▌     | 15/33 [00:13<00:15,  1.16it/s]




-(Prediction):  52%|█████▏    | 17/33 [00:14<00:13,  1

In [64]:
# Opposing labels
len(predictions[round(predictions[0].astype(float)) - round(predictions[3].astype(float)) != 0])

1501

In [67]:
# Great difference
len(predictions[abs(predictions[0] - predictions[3]) > 0.5])

212